# Cache

A Cache is an information technology for the temporary storage (caching) of Web documents, such as Web pages, images, and other types of Web multimedia, to reduce server lag.

Caching is one of those methods which a website implements to become faster. It is cost efficient and saves CPU processing time.

Django comes with a robust cache system that lets you save dynamic pages so they don't have to be calulated for each request.

You can cache the output of specific views, you can cache only the pieces that are difficult to produce, or you can cache your entire site.

Following are the options of caching:-

    > Database Caching
    
    > File System Caching
    
    > Local Memory Caching
    
## How to implement Caching

* The per-site cache - Once the cache is set up, the simplest way to use caching is to cache your entire site.

* The per-view cache - A more granular way to use the caching framework is by caching the output of individual views.

* Template fragement caching -  This gives you more control what to cache.

### The per-site cache 

The per-site cache - Once the cache is set up, the simplest way to use caching is to cache your entire site.

MIDDLEWARE = [
    
    'django.middleware.cache.UpdateCacheMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.cache.FetchFromCacheMiddleware',
]

CACHE_MIDDLEWARE_ALIS - The cache alias to use for storage.

CACHE_MIDDLEWARE_SECONDS - The number of seconds each page should be cached.

CACHE_MIDDLEWARE_KEY_PREFIX - If the cache is shared across multiple sites using the same Django installation, set this to the name of the site, or some other string that is unique to this Django instance, to prevent key collisions. Use an empty string if you don't care.

#### Database Caching

Django can store its cached data in your database. This works best if you've got a fast, well-indexed database server.

CACHES = {
    'default':{
        'BACKEND': 'django.core.cache.backends.db.DatabaseCache',
        'LOCATION': 'my_cache_table',
        'TIMEOUT' : 60, (seconds)
        'OPTIONS':{
            'MAX_ENTRIES': 1000
        }
    }
}


Before using the database cache, you must create the cache table with this command:

**python manage.py createcachetable**

This creates a table in your database that is in the proper format that Django's database-cache system expects. The name of the table is taken from LOCATION.

If you are using multiple database caches, createcachetable creates one tabel for each cache.

#### Cache Arguments

Each cache backend can be given additional arguments to control caching behaviour.

TIMEOUT: The default timeout, in seconds, to use for the cache. This argument defaults to 300 seconds (5 minutes). You can set TIMEOUT to None so that, by default, cache keys never expire. A value of 0 causes keys to immediately expire (effectively "don't cache").

OPTIONS: Any options that should be passed to the cache backend. The list of valid options will vary with each backend, and cache backends backed by a third-party library will pass their options directly to the underlying cache library.

Cache backends that implement their own culling strategy (i.e., the loemem, filesystem and database backends) will honor the following options:

MAX_ENTRIES: The maximum number of entries allowed in the cache before old values are deleted. This argument defaults to 300.

CULL_FREQUENCY: The fraction of entries that are culled when MAX_ENTRIES is reached. The actual ratio is 1/CULL_FREQUENCY, so set CULL_FREQUENCY to 2 to cull half the entries when MAX_ENTRIES is reached. This argument should be an integer and defaults to 3.

A value of 0 for CULL_FREQUENCY means that the entire cache will be dumped when MAX_ENTIRES is reached. On some backends (database in particular) this makes culling much faster at the expense of more cache misses.

## Filesystem Caching

The file-base backend serializes and stores each cache value as a separate file.

    CACHES = {
        'default':{
            'BACKEND': 'django.core.cache.backends.filebased.FileBasedCache',
            'LOCATION': 'c:/Django code/gs80',  # Absolute directory path where all cache file will be store.  
        }
    }

Make sure the directory pointed-to by this setting exists and is readable and writable by the system user under which your Web server runs.

    CACHES = {
        'default':{
            'BACKEND': 'django.core.cache.backends.filebased.FileBasedCache',
            'LOCATION': '/var/tmp/django_cache',
        }
    }
    
## Local Memory Caching

This is the default cache if another is not specified in your settings file. This cache is per-process and thread-safe.

Each process will have its own private cache instance, which means no cross-process caching is possible. This obviously also means the local memory cache isn't particularly memory-efficient.

It's probably not a good choice for production environments. It's nice for development.

    CACHES = {
        'default': 'django.core.cache.backends.locmem.LocMemCache',
        'LOCATION': 'unique-snowflake', # This cache LOCATION is used to identify individual memory stores.
    }
    
# How to implement Caching

* The per-site cache - Once the cache is set up, the simplest way to use caching is to cache your entrie site.

* The per-view cache- A more granular way to use the caching framework is by caching the ouput of individual views.

* Template fragment caching - This gives you more control what to cache.

## The per-view cache

The per-view cache - A more granular way to use the caching framework is by caching the output of individual views. django.views.decorators.cache defines a cache_page decorator that will automatically cache the view's reponse. If multiple URLs point at the same view, each URL will be cached separately.

    from django.views.decorators.cache import cache_page
    @cache_page(timeout, cache, key_prefix)
    def my_view(request):
        # body

timeout - The cache timeout, in seconds.

cache - This directs the decorator to use a specific cache (from your CACHES setting) when caching view results. By default, the default cache will be used.

Key_prefix - You can also override the cache prefix on a per-view basis. It works in the same way as the CACHE_MIDDLEWARE_KEY_PREFIX setting for the middleware.

**Specifying per-view cache in the URLconf**

    from django.views.decorators.cache import cache_page
    urlpatterns = [
        path('route/', cache_page(timeout, cache, key_prefix)(view_function)),
    ]
    
    # or 
    urlpatterns = [
        path('home/', cache_page(60)(view_function), name="home"),
    ]
    
## Template Fragment Caching

This gives you more control what to cache.

{% load cache %} - This gives access to cache tag in template

{% cache %} - This template tag cache the contents of the block for a given amount of time.

**Syntax:-**

    {% cache timeout name variable using="" %} 
    ...............
    {% endcache name %}
    
    Timeout - The cache timeout, in seconds. The fragment is cached forever if timeout is None. It can be a template variable, as long as the template variable resolves to an integer value.
    
    Name - The name to give the cache fragment. The name will be taken as is, do not use a variable. 
    
    Variable - This can be a variable with or without filters. This will cache multiple copies of a fragment depending on some dynamic data that appears inside the fragment.
    
    using - The cache tag will try to use the given cache. If no such cache exists, it will fall back to using the default cache. You may select an alternate cache backend to use with the using keyword argument, which must be the last argument to the tag.
    
**Example:-**

    {% load cache %}
    {% cache 60 sidebar request.user.username using="localcache "%}
    .......sidebar for logged in user...............
    {% endcache %}
    
# Low-Level Cache API

Sometimes, cachin an entire rendered page doesn't gain you very much and is, in fact, inconvenient overkill.

Perhaps, for instance, your site includes a view whose results depend on several expensive queries, the results of which change at different intervals. In this case, it would not be ideal to use the full-page caching that the per-site or per-view cache strategies offer, because you wouldn't want to cache the entire result (since some of the data changes often), but you'd  still want to cache the results that rarely change.

For cases like this, Django exposes a low-level cache API. You can use this API to store objects in the cache with any level of granularity you like. You can cache any Python object that can be pickled safely: strings, dictionaries, lists of model objects, and so forth.

    django.core.cache.cache
    
from django.core.cache import cache
* cache.set(key, value, timeout=DEFAULT_TIMEOUT, version=None) - This method is used to set cache.

where,

key - It should be str.

value - It can be any pickleable Python object.

timeout - It is number of seconds the value should be stored in the cache. Timeout of None will cache the value forever. A timeout of 0 won't cache the value.

version - It is an int. You can set cache with same key but different version.

* cache.get(key, default=None, version=None) - This method is used to get cache. If the key doesn't exists in the cache, it returns None.

default - This specifies which value to return if the object doesn't exist in the cache.

* cache.add(key, value, timeout=DEFAULT_TIMEOUT, version=None) - This method is used to add a key only if it doesn't already exist. It takes the same parameters as set(), but it will not attempt to update the cache if the key specified is already present. If you need to know whether add() stored a value in the cache, you can check the return value. It will return True if the value was stored, False otherwise.

* cache.get_or_set(key, default, timeout=DEFAULT_TIMEOUT, version=None) - This method is used to get a key's value or set a value if the key isn't in the cache. It takes the same parameters as get() but the default is set as the new cache value for that key, rather than returned. You can also pass any callable as a default value.

* cache.set_many(dict, timeout) - This method is used to set multiple values more efficiently, use set_many() to pass a dictionary of key-value pairs.

* cache.get_many(keys, version=None) - There's also a get_many() interface that only hits the cache once. get_many() returns a dictionary with all the keys you asked for that actually exist in the cache (and haven't expired).

* cache.delete(key, version=None) - This method is used to delete keys explicitly to clear the cache for a particular object.

* cache.delete_many(keys, version=None) - This method is used to clear a bunch of keys at once. It can take a list of keys to be cleared.


* cache.clear() - This method is used to delete all the keys in the cache. Be careful with this; clear() will remove everything from the cache, not just the keys set by your application.

* cache.touch(key, timeout=DEFAULT_TIMEOUT, version=None) - This method is used to set a new expiration for a key.touch() returns True if the key was successfully touched, False otherwise.

* cache.incr(key, delta=1, version=None)

* cache.decr(key, delta=1, version=None)

* cache.close() - You can close the connection to your cache with close() if implemented by the cache backend.